In [0]:
from pyspark.sql.functions import col, udf, row_number, lit, rand
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from math import ceil
from random import choice, randint

In [0]:
# Criando dados aleatórios
line_number = 33

data = []

for x in range(line_number):
  data.append([f'{x+randint(0, line_number)}', 'anna', 'julia'])
  
columns = ['id', 'name', 'lastname']

dataframe = spark.createDataFrame(data, columns)
display(dataframe)

id,name,lastname
11,anna,julia
14,anna,julia
14,anna,julia
5,anna,julia
21,anna,julia
18,anna,julia
9,anna,julia
24,anna,julia
36,anna,julia
23,anna,julia


In [0]:
# Ranqueando as linhas para realizar a separação pelos números
# Ordene pela coluna que deseja ou use rand() para ordenar aleatoriamente
ranked = dataframe.withColumn("row_number",row_number().over(Window.partitionBy().orderBy(rand())))
display(ranked)

id,name,lastname,row_number
39,anna,julia,1
52,anna,julia,2
34,anna,julia,3
58,anna,julia,4
14,anna,julia,5
46,anna,julia,6
31,anna,julia,7
16,anna,julia,8
24,anna,julia,9
43,anna,julia,10


#### Divisão para que grupos fiquem +- com o mesmo número de linhas
##### Podem ter infinitos grupos

In [0]:
# Verificando o ranking mais alto
max_rank = ranked.count()

# Contador de grupos, iniciando em zero
div_count = 0

# Variável que irá guardar as divisoes do ranking máximo
min_div = max_rank

# Contando quantas vezes é preciso dividir o max_rank para chegar em em aproximadamente zero
# Resultando em um número para verificar o total de linhas por grupo
# O contador indica o número de divisões ate zero do max_rank
while min_div > 0:
  min_div = int(min_div / 2)
  div_count += 1

# Então o max_rank é dividido por esse contador para verificar o máximo de linhas em cada grupo
min_lines_group = int(max_rank / div_count)

# Com um número mínimo de pessoas por grupo, é montado um vetor que contém os indices para a divisão
# Os indices são comparados com os rankings de cada linha para realizar a divisão
divisions = [x for x in range(min_lines_group, max_rank+1, min_lines_group)]

# Valor da última partição
last_partition = divisions[len(divisions)-1]

# Linhas sobrando
remaining_lines = max_rank - last_partition

# Último grupo cheio
max_group = int(last_partition / min_lines_group)

# Lista de grupos
group_list = list(range(1, max_group+1))

# Verificando números
print(f"Total de grupos: {div_count}")
print(f"Mínimo de linhas por grupo: {min_lines_group}")
print(f"Partições: {divisions}")
print(f"Última partição: {last_partition}")
print(f"Linhas Sobrando: {remaining_lines}")
print(f"Último grupo: {max_group}")
print(f"Lista de Grupos: {group_list}")

Total de grupos: 6
Mínimo de linhas por grupo: 5
Partições: [5, 10, 15, 20, 25, 30]
Última partição: 30
Linhas Sobrando: 3
Último grupo: 6
Lista de Grupos: [1, 2, 3, 4, 5, 6]

In [0]:
# Função para separar em grupos
def set_multiple_groups(rank, place_remaning=1):
  
  if place_remaning not in [1, 2, 3, 4]:
    raise Exception("Valores devem estar entre (1, 2, 3, 4)")
  
  # O ranking da linha é verificando em qual parte da divisão ele se encontra
  for partition in divisions:
    # Se ele é menor que a partição ou igual, então está nesse grupo
    if rank <= partition:
      # Retornando núemro do grupo
      # Dividindo a particao pelo máximo de linhas por grupo, ficando em sequência (Grupo 1, 2, 3, ...)
      return f"Grupo { int ( partition / min_lines_group ) }"
  
  # Caso o ranking seja maior que ultima partição ele entraria em um novo ultimo grupo
  # Mas para equalizar o número de linhas por grupo, as ultimas linhas serão distribuidas nos grupos já existentes
  
  # Diferença do valor atual com a ultima partição
  current_diff = rank - last_partition
  
  if place_remaning == 1:
    
    # Linhas restantes nos primeiros grupos
    # A primeira linha restante vai para o grupo 1, a segunda linha restante vai para o grupo 2, e assim continua
    # O grupo é definido pela diferença do ranking atual com o valor da ultima partição
    return f"Grupo { rank - last_partition }"
  
  elif place_remaning == 2:
    
    # Linhas restantes nos últimos grupos
    # A última linha sobrando irá para o último grupo definido, a penúltima linha irá para o penúltimo grupo criado e assim continua
    # O grupo é definido pela diferença do grupo de maior número com as quantidade de linhas sobrando, somando a diferença do ranking atual com a última partição
    return f"Grupo { max_group - remaining_lines + current_diff }"
  
  elif place_remaning == 3:
    
    # Linhas restantes nos últimos grupos
    # A primeira linha sobrando irá para o último grupo definido, a segunda linha irá para o penúltimo grupo criado e assim continua
    # O grupo é definido pela diferença do grupo de maior número com a diferença do valor atual com a ultima partição somando 1
    return f"Grupo { max_group - current_diff + 1 }"
  
  elif place_remaning == 4:
    
    # Linhas restantes nos em grupos aleatórios
    # Para cada linha restante, é sorteado um grupo
    
    # Busca um grupo aleatório dos disponíveis
    random_num = choice(group_list)
    
    # Retira o grupo sorteado da lista para que a distribuição fique equilibrada
    group_list.remove(random_num)

    return f"Grupo { random_num }"

In [0]:
# Função para usar em colunas do df
set_multiple_groups_udf = udf(set_multiple_groups, StringType())

In [0]:
grouped = ranked.withColumn("group", set_multiple_groups_udf(col("row_number"), lit(4)))
display(grouped)

id,name,lastname,row_number,group
39,anna,julia,1,Grupo 1
52,anna,julia,2,Grupo 1
34,anna,julia,3,Grupo 1
58,anna,julia,4,Grupo 1
14,anna,julia,5,Grupo 1
46,anna,julia,6,Grupo 2
31,anna,julia,7,Grupo 2
16,anna,julia,8,Grupo 2
24,anna,julia,9,Grupo 2
43,anna,julia,10,Grupo 2


#### Divisão com um número máximo de grupos
##### Podem ter infintas linhas por grupo

In [0]:
# Verificando o ranking mais alto
max_rank = ranked.count()

# Definindo o número máximo de grupos
max_group = 10

# Mínimo de linhas por grupo
min_lines_group = int(max_rank / max_group)
if min_lines_group == 0:
  min_lines_group = 1

# Valor da última partição
last_partition = (max_group * min_lines_group)

# Com um número mínimo de pessoas por grupo, é montado um vetor que contém os indices para a divisão
# Os indices são comparados com os rankings de cada linha para realizar a divisão
divisions = [x for x in range(min_lines_group, last_partition + 1, min_lines_group)]

# Linhas sobrando
remaining_lines = max_rank - last_partition

# Lista de grupos
group_list = list(range(1, max_group+1))

print(f"Total de grupos: {max_group}")
print(f"Mínimo de linhas por grupo: {min_lines_group}")
print(f"Partições: {divisions}")
print(f"Última partição: {last_partition}")
print(f"Linhas Sobrando: {remaining_lines}")
print(f"Último grupo: {max_group}")
print(f"Lista de Grupos: {group_list}")

Total de grupos: 10
Mínimo de linhas por grupo: 3
Partições: [3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
Última partição: 30
Linhas Sobrando: 3
Último grupo: 10
Lista de Grupos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [0]:
# Função para separar em grupos
def set_group_divisions(rank, place_remaning=1):
  
  if place_remaning not in [1, 2, 3, 4]:
    raise Exception("Valores devem estar entre (1, 2, 3, 4)")
  
  # O ranking da linha é enviado
  # Verificando em qual parte da divisão ele se encontra
  for partition in divisions:
    # Se ele é menor que a partição ou igual, então está nesse grupo
    if rank <= partition:
      # Retornando núemro do grupo
      # Dividindo a particao pelo máximo de linhas por grupo, ficando em sequeincia (Grupo 1, 2, 3, ...)
      return f"Grupo { int ( partition / min_lines_group ) }"
  
  # Caso o ranking seja maior que ultima partição ele entraria em um novo ultimo grupo
  # Mas para equalizar o número de linhas por grupo, as ultimas linhas serão distribuidas nos grupos já existentes
  
  # Diferença do valor atual com a ultima partição
  current_diff = rank - last_partition
  
  if place_remaning == 1:
    
    return f"Grupo { rank - last_partition }"
  
  elif place_remaning == 2:
    
    # Linhas restantes nos últimos grupos
    # A última linha sobrando irá para o último grupo definido, a penúltima linha irá para o penúltimo grupo criado e assim continua
    # O grupo é definido pela diferença do grupo de maior número com as quantidade de linhas sobrando, somando a diferença do ranking atual com a última partição
    return f"Grupo { max_group - remaining_lines + current_diff }"
  
  elif place_remaning == 3:
    
    # Linhas restantes nos últimos grupos
    # A primeira linha sobrando irá para o último grupo definido, a segunda linha irá para o penúltimo grupo criado e assim continua
    # O grupo é definido pela diferença do grupo de maior número com a diferença do valor atual com a ultima partição somando 1
    return f"Grupo { max_group - current_diff + 1 }"
  
  elif place_remaning == 4:
    
    # Linhas restantes nos em grupos aleatórios
    # Para cada linha restante, é sorteado um grupo
    
    # Busca um grupo aleatório dos disponíveis
    random_num = choice(group_list)
    
    # Retira o grupo sorteado da lista para que a distribuição fique equilibrada
    group_list.remove(random_num)

    return f"Grupo { random_num }"

In [0]:
# Função para usar em colunas do df
set_group_divisions_udf = udf(set_group_divisions, StringType())

In [0]:
grouped_2 = ranked.withColumn("group", set_group_divisions_udf(col("row_number"), lit(4)))
display(grouped_2)

id,name,lastname,row_number,group
39,anna,julia,1,Grupo 1
52,anna,julia,2,Grupo 1
34,anna,julia,3,Grupo 1
58,anna,julia,4,Grupo 2
14,anna,julia,5,Grupo 2
46,anna,julia,6,Grupo 2
31,anna,julia,7,Grupo 3
16,anna,julia,8,Grupo 3
24,anna,julia,9,Grupo 3
43,anna,julia,10,Grupo 4
